# Experimento mestrado 1
## Executando na pasta principal

In [1]:
import os
import numpy as np
os.chdir('../../../') #Executando na pasta Visual_Memory
import sys
sys.path.append('./include')
sys.path.append('./src')

## Copilando arquivos para teste

In [2]:
if os.system("cd Workbench/ && ./convertendo_all.sh && cd ..") != 0:
    print "Erro de copilação"
    sys.exit(0)

## Criando configurações padroes e arquivo de execução

In [3]:
ERRO = 0.1

os.system('sed -i \'s/time.time( )/self._bkb.read_float("VISION_RB20_TAG")/g\' ./include/KalmanFilter.py')
os.system('sed -i \'s/time.time( )/self._bkb.read_float("VISION_RB20_TAG")/g\' ./src/Behavior.py')
os.system('sed -i \'/time.sleep(delta)/d\' ./src/Behavior.py')
os.system('sed -i \'s/if delta > 0:/self._bkb.write_float("VISION_RB21_TAG", 0)\\n                while self._bkb.read_float("VISION_RB21_TAG") == 0:\\n                    time.sleep(0.5)/g\' ./src/Behavior.py')

text = open("./run.sh", "w")
text.write("#!/bin/bash\n\npython visualMemory.py --nr 3 --p 2000 --d")
text.close()

In [4]:
from Blackboard import *
import random

In [5]:
bkb = Blackboard( )

In [6]:
bkb.write_float('VISION_RB20_TAG', 0) # tempo no now
bkb.write_float('VISION_RB21_TAG', 0) # Parando visual 

# Zerando robôs
bkb.write_float('VISION_RB01_TAG', 0)
bkb.write_float('VISION_RB02_TAG', 0)

bkb.write_float('VISION_LAND_TAG', 0)

bkb.write_float("VISUAL_MEMORY_LAND_LOC", 0)
bkb.write_float("VISUAL_MEMORY_LAND_X", 0)
bkb.write_float("VISUAL_MEMORY_LAND_Y", 0)

bkb.write_float("VISUAL_MEMORY_AL_01_LOC", 0)
bkb.write_float("VISUAL_MEMORY_AL_01_X", 0)
bkb.write_float("VISUAL_MEMORY_AL_01_Y", 0)
bkb.write_float("VISUAL_MEMORY_AL_01_MAX_VEL", 0)

bkb.write_float("VISUAL_MEMORY_OP_01_LOC", 0)
bkb.write_float("VISUAL_MEMORY_OP_01_X", 0)
bkb.write_float("VISUAL_MEMORY_OP_01_Y", 0)
bkb.write_float("VISUAL_MEMORY_OP_01_MAX_VEL", 0)

## Executando VisualMemory e removendo arquivo de execução

In [7]:
os.system("chmod 777 run.sh")
os.system("kill $(pidof -x ./run.sh)")
os.system("screen -d -m -S 'VisualMemory' './run.sh'")
os.system("gnome-terminal -x sh -c 'screen -R VisualMemory'")
os.system("rm run.sh")

0

In [ ]:
FALHA = 50
FALHACOR = 100
# vx = 2; vy = 1; vax = 5; vay = -1; vox = -3; voy = -1.5; ω = 1°
listaLand = [
    {"tag": 1, "pos": [+6.00, +3.00], "movement": 1, "time": 0.0},
    {"tag": 1, "pos": [+5.03, +2.45], "movement": 1, "time": 0.5},
    {"tag": 1, "pos": [+4.05, +1.89], "movement": 1, "time": 1.0},
    {"tag": 1, "pos": [+3.08, +1.34], "movement": 1, "time": 1.5},
    {"tag": 1, "pos": [+2.10, +0.79], "movement": 1, "time": 2.0},
    {"tag": 1, "pos": [+1.13, +0.24], "movement": 1, "time": 2.5},
    {"tag": 1, "pos": [+0.15, -0.32], "movement": 1, "time": 3.0},
    {"tag": 1, "pos": [-0.83, -0.87], "movement": 1, "time": 3.5},
    {"tag": 1, "pos": [-1.81, -1.43], "movement": 1, "time": 4.0},
    {"tag": 1, "pos": [-2.78, -1.98], "movement": 1, "time": 4.5},
    {"tag": 1, "pos": [-3.76, -2.53], "movement": 1, "time": 5.0},
    {"tag": 1, "pos": [-4.74, -3.09], "movement": 1, "time": 5.5},
    {"tag": 1, "pos": [-5.72, -3.64], "movement": 1, "time": 6.0},
    {"tag": 1, "pos": [-6.70, -4.20], "movement": 1, "time": 6.5},
    {"tag": 1, "pos": [-7.68, -4.75], "movement": 1, "time": 7.0},
    {"tag": 1, "pos": [-8.66, -5.31], "movement": 1, "time": 7.5},
    {"tag": 1, "pos": [-9.64, -5.86], "movement": 1, "time": 8.0},
]

listaOponente = [
    {"tag": 1, "pos": [+7.00, +0.00], "movement": 1, "time": 0.0},
    {"tag": 1, "pos": [+4.49, -1.30], "movement": 1, "time": 0.5},
    {"tag": 1, "pos": [+1.97, -2.57], "movement": 1, "time": 1.0},
    {"tag": 1, "pos": [-0.56, -3.81], "movement": 1, "time": 1.5},
    {"tag": 1, "pos": [-3.11, -5.03], "movement": 1, "time": 2.0},
    {"tag": 1, "pos": [-5.66, -6.22], "movement": 1, "time": 2.5},
    {"tag": 1, "pos": [-8.23, -7.39], "movement": 1, "time": 3.0},
    {"tag": 1, "pos": [-10.8, -8.53], "movement": 1, "time": 3.5},
    {"tag": 1, "pos": [-13.4, -9.64], "movement": 1, "time": 4.0},
    {"tag": 1, "pos": [-16.0, -10.7], "movement": 1, "time": 4.5},
    {"tag": 1, "pos": [-18.6, -11.8], "movement": 1, "time": 5.0},
    {"tag": 1, "pos": [-21.3, -12.8], "movement": 1, "time": 5.5},
    {"tag": 1, "pos": [-23.9, -13.8], "movement": 1, "time": 6.0},
    {"tag": 1, "pos": [-26.5, -14.7], "movement": 1, "time": 6.5},
    {"tag": 1, "pos": [-29.1, -15.7], "movement": 1, "time": 7.0},
    {"tag": 1, "pos": [-31.8, -16.6], "movement": 1, "time": 7.5},
    {"tag": 1, "pos": [-34.5, -17.5], "movement": 1, "time": 8.0},
]

listaAliado = [
    {"tag": 3, "pos": [+6.00, +0.00], "movement": 1, "time": 0.0},
    {"tag": 3, "pos": [+4.00, -1.04], "movement": 1, "time": 0.5},
    {"tag": 3, "pos": [+1.98, -1.97], "movement": 1, "time": 1.0},
    {"tag": 3, "pos": [-0.04, -3.00], "movement": 1, "time": 1.5},
    {"tag": 3, "pos": [-2.07, -4.07], "movement": 1, "time": 2.0},
    {"tag": 3, "pos": [-4.11, -5.04], "movement": 1, "time": 2.5},
    {"tag": 3, "pos": [-6.16, -6.00], "movement": 1, "time": 3.0},
    {"tag": 3, "pos": [-8.21, -6.93], "movement": 1, "time": 3.5},
    {"tag": 3, "pos": [-10.3, -7.85], "movement": 1, "time": 4.0},
    {"tag": 3, "pos": [-12.3, -8.75], "movement": 1, "time": 4.5},
    {"tag": 3, "pos": [-14.4, -9.63], "movement": 1, "time": 5.0},
    {"tag": 3, "pos": [-16.5, -10.5], "movement": 1, "time": 5.5},
    {"tag": 3, "pos": [-18.6, -11.3], "movement": 1, "time": 6.0},
    {"tag": 3, "pos": [-20.7, -12.2], "movement": 1, "time": 6.5},
    {"tag": 3, "pos": [-22.8, -13.0], "movement": 1, "time": 7.0},
    {"tag": 3, "pos": [-24.9, -13.8], "movement": 1, "time": 7.5},
    {"tag": 3, "pos": [-27.0, -14.5], "movement": 1, "time": 8.0},
]


for lista in [listaLand, listaOponente, listaAliado]:
    for data in lista:
        if random.random() <= FALHA/100.0:
            data["pos"][0] += random.uniform(-ERRO, ERRO)
            data["pos"][1] += random.uniform(-ERRO, ERRO)
        else:
            lista.remove(data)

for lista in [listaOponente, listaAliado]:
    for data in lista:
        if random.random() > FALHACOR/100.0:
            data["tag"] = 2

In [ ]:
raw_input("Pronto para iniciar experimentos")
for t in np.arange(0, 8.5, 0.5):
    if listaLand != [] and t == listaLand[0]["time"]:
        dataLand = listaLand.pop(0)
    else:
        dataLand = None
    
    if listaOponente != [] and t == listaOponente[0]["time"]:
        dataOponente = listaOponente.pop(0)
    else:
        dataOponente = None
    
    if listaAliado != [] and t == listaAliado[0]["time"]:
        dataAliado = listaAliado.pop(0)
    else:
        dataAliado = None
    
    print dataLand
    print dataAliado
    print dataOponente
    print '\n'
    
    # Instante atual no tempo
    bkb.write_float('VISION_RB20_TAG', t + 0.5)
    
    if dataLand != None:
        bkb.write_float('VISION_LAND_X', dataLand['pos'][0])
        bkb.write_float('VISION_LAND_Y', dataLand['pos'][1])
        bkb.write_float('VISION_LAND_TIME', dataLand['time'] + 0.5)        
        bkb.write_float('VISION_LAND_MOV', dataLand['movement'])
        bkb.write_float('VISION_LAND_TAG', dataLand['tag'])
    
    if dataOponente != None:
        bkb.write_float('VISION_RB01_X', dataOponente['pos'][0])
        bkb.write_float('VISION_RB01_Y', dataOponente['pos'][1])
        bkb.write_float('VISION_RB01_TIME', dataOponente['time'] + 0.5)        
        bkb.write_float('VISION_RB01_MOV', dataOponente['movement'])
        bkb.write_float('VISION_RB01_TAG', dataOponente['tag'])
    
    if dataAliado != None:
        bkb.write_float('VISION_RB02_X', dataAliado['pos'][0])
        bkb.write_float('VISION_RB02_Y', dataAliado['pos'][1])
        bkb.write_float('VISION_RB02_TIME', dataAliado['time'] + 0.5)        
        bkb.write_float('VISION_RB02_MOV', dataAliado['movement'])
        bkb.write_float('VISION_RB02_TAG', dataAliado['tag'])
    
    # Iniciando
    raw_input("Pronto para enviar")
    bkb.write_float('VISION_RB21_TAG', 1)
    
    while bkb.read_float('VISION_RB21_TAG'):
        pass
    
    bkb.write_float('VISION_RB21_TAG', 1)
    
    while bkb.read_float('VISION_RB21_TAG'):
        pass
    
    raw_input("Finalizado\n")